# Assignment: Preparing Data for Analysis (Modified Titanic)

![](https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/%20files/hw.png)

In [82]:
#Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis, skew

In [83]:
df = pd.read_csv('https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/hw/titanic_training_dataset_v2.csv')

In [84]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0,PC 17599,71.2833,C85,C
2,3,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0,113803,53.1000,C123,S
4,5,0.0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0,373450,8.0500,NaN,S


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     865 non-null    float64
 2   Pclass       828 non-null    float64
 3   Name         866 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        858 non-null    float64
 7   Parch        891 non-null    int64  
 8   Ticket       852 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     803 non-null    object 
dtypes: float64(5), int64(2), object(5)
memory usage: 83.7+ KB


# 1) Load data & review the data

<font color='blue'>Q1) What is the shape of this dataset? (rows & columns)</font>

In [86]:
df.shape

(891, 12)

# 2) Drop unqualified variables

*   Drop variables with missing > 50%
*   Drop categorical variables with flat values > 70% (variables with the same value in the same column)

<font color='blue'>Q2) How many columns do we have left?</font>




10

In [87]:
half_count = len(df) / 2
sev_ty_per = len(df)*0.7

In [88]:
df = df.dropna(thresh = half_count,axis = 1)
for i in df.columns:
  if(df[i].value_counts().max() > sev_ty_per):
    df = df.drop(columns=[i])
df.shape

(891, 10)

# 3) Remove all rows with missing target (the variable "Survived")

<font color='blue'>Q3) How many rows do we have left?</font>

865

In [89]:
null_counts = df['Survived'].isnull().sum()
print("Number of null values in Survied column:\n{}".format(null_counts))

Number of null values in Survied column:
26


In [90]:
df = df.dropna(subset=['Survived'])

In [91]:
df.shape

(865, 10)

# 4) Handle outliers 

For the variable “Fare”, replace outlier values with the boundary values


*   If value < (Q1 - 1.5IQR), relace with (Q1 - 1.5IQR)
*   If value > (Q3 + 1.5IQR), relace with (Q3 + 1.5IQR)

<font color='blue'>Q4) What is the mean of “Fare” after replacing the outliers (round 2 decimal points)?</font>



ans:24.04

In [92]:
q75, q25 = np.percentile(df.Fare, [75 ,25])
iqr = q75 - q25

min = q25 - (iqr*1.5)
max = q75 + (iqr*1.5)

In [93]:
df["Fare"] = df["Fare"].where(df["Fare"] >= min, min)
df["Fare"] = df["Fare"].where(df["Fare"] <= max, max)

In [94]:
df['Fare'].mean()

24.038996878612718

# 5) Impute missing value



*   Show the number of missing values in each variable
*   Impute missing values with mean & mode
*   Show the number of missing values again after missing value imputation

<font color='blue'>Q5) Which variable has the largest number of missing values?</font>



Age has largest number of missing values : 172

In [95]:
for i in df.columns:
    print(i,"has null = ", df[i].isnull().sum())

PassengerId has null =  0
Survived has null =  0
Pclass has null =  63
Name has null =  25
Sex has null =  0
Age has null =  172
SibSp has null =  33
Ticket has null =  39
Fare has null =  0
Embarked has null =  87


# 6) Convert categorical to numeric values



*   Drop the variables “Name” & “Ticket”
*   For the variables “Sex” & “Embarked”, perform the dummy coding and drop the first level. Also, drop those original variables (“Sex” & “Embarked”)

<font color='blue'>Q6) How many columns do we have?</font>



Columns = 9

In [96]:
df = df.drop(["Name","Ticket"], axis = 1)
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Fare',
       'Embarked'],
      dtype='object')

In [97]:
df_dummies = pd.get_dummies(df[["Sex","Embarked"]], drop_first=True)

In [98]:
df_dummies

,Sex_male,Embarked_Q,Embarked_S
0,1,0,1
1,0,0,0
2,0,0,1
3,0,0,1
4,1,0,1
...,...,...,...
886,1,0,1
887,0,0,1
888,0,0,1
889,1,0,0


In [99]:
df = pd.concat([df,df_dummies], axis = 1)

In [100]:
df = df.drop(["Sex","Embarked"], axis = 1)

In [101]:
df

,PassengerId,Survived,Pclass,Age,SibSp,Fare,Sex_male,Embarked_Q,Embarked_S
0,1,0.0,3.0,22.0,1.0,7.2500,1,0,1
1,2,1.0,1.0,38.0,1.0,65.6563,0,0,0
2,3,1.0,3.0,26.0,0.0,7.9250,0,0,1
3,4,1.0,1.0,35.0,1.0,53.1000,0,0,1
4,5,0.0,3.0,35.0,0.0,8.0500,1,0,1
...,...,...,...,...,...,...,...,...,...
886,887,0.0,2.0,27.0,0.0,13.0000,1,0,1
887,888,1.0,1.0,19.0,0.0,30.0000,0,0,1
888,889,0.0,3.0,NaN,1.0,23.4500,0,0,1
889,890,1.0,1.0,26.0,0.0,30.0000,1,0,0


In [102]:
df.shape

(865, 9)

# 7) Partition data



*   Split train/test split with stratification using 70%:30% and random seed with 12345
*   Show a proportion between survived (1) and died (0) in all data sets (total data, train, test)

<font color='blue'>Q7) What is a proportion between survived (1) and died (0) in the training data?</font>



Train_data survived =  0.3834710743801653

Train_data survived =  0.6165289256198347

In [103]:
from sklearn.model_selection import train_test_split

In [104]:
X = df
y = df['Survived']

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=12345)

In [106]:
survived_total = (df["Survived"] == 1).sum()
print("Total_data survived = ",survived_total/df.shape[0])
dead_total = (df["Survived"] == 0).sum()
print("Total_data survived = ",dead_total/df.shape[0])

Total_data survived =  0.3838150289017341
Total_data survived =  0.6161849710982659


In [107]:
survived_train = (y_train == 1).sum()
print("Train_data survived = ",survived_train/y_train.shape[0])
dead_train = (y_train == 0).sum()
print("Train_data survived = ",dead_train/y_train.shape[0])

Train_data survived =  0.3834710743801653
Train_data survived =  0.6165289256198347


In [108]:
survived_test = (y_test == 1).sum()
print("Test_data survived = ",survived_test/y_test.shape[0])
dead_test = (y_test == 0).sum()
print("Test_data survived = ",dead_test/y_test.shape[0])

Test_data survived =  0.38461538461538464
Test_data survived =  0.6153846153846154


In [109]:
print("Row in training set = ", X_train.shape[0])
print("Row in teat set = ", X_test.shape[0])

Row in training set =  605
Row in teat set =  260
